In [107]:
import sys
from pathlib import Path
import pandas as pd 
import numpy as np
import keras
import math 
import tensorflow as tf
from keras.layers import LSTM, Dense, Dropout, LeakyReLU, GRU, BatchNormalization, Input, LayerNormalization
from keras.regularizers import l1, l2, l1_l2
from tensorflow.keras.optimizers import SGD
import tensorflow_addons as tfa
from keras.models import Sequential
from keras.callbacks import LearningRateScheduler, EarlyStopping
from keras.preprocessing.sequence import TimeseriesGenerator
sys.path.append(str(Path("../..").resolve()))
from src.constants import model_data_dir, data_dir
from src.utils import use_target

In [57]:
df_train = pd.read_csv(model_data_dir / "train_classification.csv")
df_test = pd.read_csv(model_data_dir / "test_classification.csv")
test_dates = pd.to_datetime(df_test["date"])
df_train = df_train.drop(columns=["date"])
df_test = df_test.drop("date", axis=1)
df_train = df_train.loc[:, ~df_train.columns.str.contains("_mv_")]
df_test = df_test.loc[:, ~df_test.columns.str.contains("_mv_")]
df_train = pd.get_dummies(df_train.astype({
    "month": "category",
    "weekday": "category",
    "day": "category",
}), drop_first=True)
df_test = pd.get_dummies(df_test.astype({
    "month": "category",
    "weekday": "category",
    "day": "category",
}), drop_first=True)

## LSTM

In [60]:
win_length = 40
batch_size = 10
num_features = df_train.shape[1] - 3

In [140]:
model = Sequential()
model.add(Input(shape=(win_length, num_features)))
# model.add(LSTM(64, return_sequences=True, bias_initializer="zeros", unit_forget_bias=True, kernel_regularizer=l1(1e-4), recurrent_regularizer=l2(2e-4)))
# model.add(LeakyReLU(alpha=0.5)) 
# model.add(LayerNormalization())
# model.add(Dropout(0.3)) 
model.add(LSTM(20, return_sequences=False, bias_initializer="zeros", unit_forget_bias=True, kernel_regularizer=l1(1e-4), recurrent_regularizer=l2(2e-4)))
model.add(LeakyReLU(alpha=0.5)) 
model.add(LayerNormalization())
model.add(Dropout(0.3)) 
model.add(Dense(1, activation="sigmoid", kernel_regularizer=l1_l2(1e-4, 2e-4)))

In [141]:
early_stop = EarlyStopping(monitor = "loss",
                           patience = 5)

step = tf.Variable(0, trainable=False)
schedule = tf.optimizers.schedules.PiecewiseConstantDecay(
    [10000, 15000], [1e-0, 1e-1, 1e-2])
# lr and wd can be a function or a tensor
lr = 1e-5 * schedule(step)
wd = lambda: 1e-4 * schedule(step)
model.compile(loss=keras.losses.binary_crossentropy,
            #   optimizer=tfa.optimizers.SGDW(
            #   learning_rate=lr, 
            #   weight_decay=wd, 
            #   momentum=0.9),
              optimizer=tfa.optimizers.AdamW(learning_rate=lr, weight_decay=wd), 
              metrics=["accuracy"])

## size

In [142]:
y_train_size, x_train_size = use_target(df_train, "sc_1d_fwd_rel_d", "classification")
y_test_size, x_test_size = use_target(df_test, "sc_1d_fwd_rel_d", "classification")

train_generator_size = TimeseriesGenerator(x_train_size, y_train_size, length=win_length, sampling_rate=1, batch_size=batch_size)
test_generator_size = TimeseriesGenerator(x_test_size, y_test_size, length=win_length, sampling_rate=1, batch_size=batch_size)

In [143]:
history = model.fit(train_generator_size, 
                    epochs=50,
                    validation_data=test_generator_size,
                    shuffle=False,
                    callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)])

Epoch 1/50
334/334 [==============================] - 7s 8ms/step - loss: 1.1433 - accuracy: 0.5094 - val_loss: 1.1193 - val_accuracy: 0.5130
Epoch 2/50
334/334 [==============================] - 2s 7ms/step - loss: 1.0558 - accuracy: 0.5010 - val_loss: 1.0578 - val_accuracy: 0.5152
Epoch 3/50
334/334 [==============================] - 2s 7ms/step - loss: 0.9912 - accuracy: 0.5010 - val_loss: 1.0083 - val_accuracy: 0.5168
Epoch 4/50
334/334 [==============================] - 2s 7ms/step - loss: 0.9433 - accuracy: 0.5115 - val_loss: 0.9686 - val_accuracy: 0.5163
Epoch 5/50
334/334 [==============================] - 2s 7ms/step - loss: 0.9069 - accuracy: 0.5094 - val_loss: 0.9338 - val_accuracy: 0.5173
Epoch 6/50
334/334 [==============================] - 2s 7ms/step - loss: 0.8890 - accuracy: 0.5043 - val_loss: 0.9055 - val_accuracy: 0.5179
Epoch 7/50
334/334 [==============================] - 2s 6ms/step - loss: 0.8569 - accuracy: 0.5073 - val_loss: 0.8804 - val_accuracy: 0.5163
Epoch 

In [144]:
size_preds = np.where(model.predict(test_generator_size).flatten() > 0.5, 1, 0)
true = y_test_size[0:len(y_test_size) - win_length ]
np.mean(np.where(size_preds > 0.5, 1, 0) == true)

0.5173347778981582

## momentum

In [146]:
y_train_mom, x_train_mom = use_target(df_train, "mom_1d_fwd_rel_d", "classification")
y_test_mom, x_test_mom = use_target(df_test, "mom_1d_fwd_rel_d", "classification")

train_generator_mom = TimeseriesGenerator(x_train_mom, y_train_mom, length=win_length, sampling_rate=1, batch_size=batch_size)
test_generator_mom = TimeseriesGenerator(x_test_mom, y_test_mom, length=win_length, sampling_rate=1, batch_size=batch_size)

In [147]:
model = Sequential()
model.add(Input(shape=(win_length, num_features)))
model.add(LSTM(20, bias_initializer="zeros", unit_forget_bias=True, kernel_regularizer=l1(1e-4), recurrent_regularizer=l2(2e-4)))
model.add(LeakyReLU(alpha=0.5)) 
model.add(LayerNormalization())
model.add(Dropout(0.3)) 
model.add(Dense(1, activation="sigmoid", kernel_regularizer=l1_l2(1e-4, 2e-4)))

In [148]:
early_stop = EarlyStopping(monitor = "loss",
                           patience = 5)

step = tf.Variable(0, trainable=False)
schedule = tf.optimizers.schedules.PiecewiseConstantDecay(
    [10000, 15000], [1e-0, 1e-1, 1e-2])
# lr and wd can be a function or a tensor
lr = 1e-5 * schedule(step)
wd = lambda: 1e-4 * schedule(step)
model.compile(loss=keras.losses.binary_crossentropy,
            #   optimizer=tfa.optimizers.SGDW(
            #   learning_rate=lr, 
            #   weight_decay=wd, 
            #   momentum=0.9),
              optimizer=tfa.optimizers.AdamW(learning_rate=lr, weight_decay=wd), 
              metrics=["accuracy"])

In [149]:
history = model.fit(train_generator_mom, 
                    epochs=50,
                    validation_data=test_generator_mom,
                    shuffle=False,
                    callbacks=[early_stop])

Epoch 1/50
334/334 [==============================] - 6s 8ms/step - loss: 0.9868 - accuracy: 0.4969 - val_loss: 0.8907 - val_accuracy: 0.5098
Epoch 2/50
334/334 [==============================] - 2s 6ms/step - loss: 0.9485 - accuracy: 0.4909 - val_loss: 0.8702 - val_accuracy: 0.5027
Epoch 3/50
334/334 [==============================] - 2s 6ms/step - loss: 0.9034 - accuracy: 0.5037 - val_loss: 0.8535 - val_accuracy: 0.4984
Epoch 4/50
334/334 [==============================] - 2s 6ms/step - loss: 0.8774 - accuracy: 0.5106 - val_loss: 0.8401 - val_accuracy: 0.4940
Epoch 5/50
334/334 [==============================] - 2s 7ms/step - loss: 0.8554 - accuracy: 0.5082 - val_loss: 0.8290 - val_accuracy: 0.4962
Epoch 6/50
334/334 [==============================] - 2s 7ms/step - loss: 0.8379 - accuracy: 0.5055 - val_loss: 0.8201 - val_accuracy: 0.4924
Epoch 7/50
334/334 [==============================] - 2s 7ms/step - loss: 0.8186 - accuracy: 0.5118 - val_loss: 0.8126 - val_accuracy: 0.4913
Epoch 

In [150]:
mom_preds = np.where(model.predict(test_generator_mom).flatten(), 1, 0)
true = y_test_mom[0:len(y_test_mom) - win_length ]
np.mean(mom_preds == true)

0.5449620801733478

## value

In [151]:
y_train_value, x_train_value = use_target(df_train, "value_1d_fwd_rel_d", "classification")
y_test_value, x_test_value = use_target(df_test, "value_1d_fwd_rel_d", "classification")

train_generator_value = TimeseriesGenerator(x_train_value, y_train_value, length=win_length, sampling_rate=1, batch_size=batch_size)
test_generator_value = TimeseriesGenerator(x_test_value, y_test_value, length=win_length, sampling_rate=1, batch_size=batch_size)

In [152]:
model = Sequential()
model.add(Input(shape=(win_length, num_features)))
model.add(LSTM(20, return_sequences=False, bias_initializer="zeros", unit_forget_bias=True, kernel_regularizer=l1(1e-4), recurrent_regularizer=l2(2e-4)))
model.add(LeakyReLU(alpha=0.5)) 
model.add(LayerNormalization())
model.add(Dropout(0.3)) 
model.add(Dense(1, activation="sigmoid", kernel_regularizer=l1_l2(1e-4, 2e-4)))

In [153]:
early_stop = EarlyStopping(monitor = "loss",
                           patience = 5)

step = tf.Variable(0, trainable=False)
schedule = tf.optimizers.schedules.PiecewiseConstantDecay(
    [10000, 15000], [1e-0, 1e-1, 1e-2])
# lr and wd can be a function or a tensor
lr = 1e-5 * schedule(step)
wd = lambda: 1e-4 * schedule(step)
model.compile(loss=keras.losses.binary_crossentropy,
            #   optimizer=tfa.optimizers.SGDW(
            #   learning_rate=lr, 
            #   weight_decay=wd, 
            #   momentum=0.9),
              optimizer=tfa.optimizers.AdamW(learning_rate=lr, weight_decay=wd), 
              metrics=["accuracy"])

In [154]:
history = model.fit(train_generator_mom, 
                    epochs=50,
                    validation_data=test_generator_mom,
                    shuffle=False,
                    callbacks=[early_stop])

Epoch 1/50
334/334 [==============================] - 6s 9ms/step - loss: 0.9784 - accuracy: 0.5142 - val_loss: 1.0125 - val_accuracy: 0.4670
Epoch 2/50
334/334 [==============================] - 2s 7ms/step - loss: 0.9399 - accuracy: 0.5103 - val_loss: 0.9907 - val_accuracy: 0.4670
Epoch 3/50
334/334 [==============================] - 2s 7ms/step - loss: 0.9096 - accuracy: 0.5055 - val_loss: 0.9695 - val_accuracy: 0.4680
Epoch 4/50
334/334 [==============================] - 2s 6ms/step - loss: 0.8938 - accuracy: 0.5103 - val_loss: 0.9508 - val_accuracy: 0.4664
Epoch 5/50
334/334 [==============================] - 2s 6ms/step - loss: 0.8767 - accuracy: 0.5043 - val_loss: 0.9337 - val_accuracy: 0.4637
Epoch 6/50
334/334 [==============================] - 2s 6ms/step - loss: 0.8573 - accuracy: 0.5127 - val_loss: 0.9181 - val_accuracy: 0.4615
Epoch 7/50
334/334 [==============================] - 2s 6ms/step - loss: 0.8386 - accuracy: 0.5106 - val_loss: 0.9032 - val_accuracy: 0.4626
Epoch 

In [155]:
value_preds = np.where(model.predict(test_generator_value).flatten() > 0.5, 1, 0)
true = y_test_value[0:len(y_test_value) - win_length ]
np.mean(value_preds == true)

0.5092091007583965

In [ ]:
pd.DataFrame({
    ""
})

In [156]:
rows = df_test.shape[0]

lstm_preds = pd.DataFrame({
    "sc": df_test["sc_1d_fwd_rel_d"][:rows-win_length], 
    "mom": df_test["mom_1d_fwd_rel_d"][:rows-win_length],
    "value": df_test["value_1d_fwd_rel_d"][:rows-win_length],
    ".pred_sc_lstm": size_preds,
    ".pred_mom_lstm": mom_preds,
    ".pred_value_lstm": value_preds
})

lstm_preds.to_csv(data_dir / "pred" / "lstm_preds.csv", index=False)

## GRU 

In [109]:
model_gru = Sequential()
model_gru.add(Input(shape=(win_length, num_features)))
model_gru.add(GRU(75, return_sequences=True))
model_gru.add(Dropout(0.3))
model_gru.add(GRU(units=30, return_sequences=True))
model_gru.add(GRU(units=30, return_sequences=False))
model_gru.add(Dropout(0.3))
model_gru.add(Dense(units=1, activation="sigmoid"))


In [110]:
early_stop = EarlyStopping(monitor = "val_loss",
                           patience = 5)

step = tf.Variable(0, trainable=False)
schedule = tf.optimizers.schedules.PiecewiseConstantDecay(
    [10000, 15000], [1e-0, 1e-1, 1e-2])
# lr and wd can be a function or a tensor
lr = 1e-1 * schedule(step)
wd = lambda: 1e-4 * schedule(step)
model_gru.compile(loss=keras.losses.binary_crossentropy,
              optimizer=tfa.optimizers.SGDW(
              learning_rate=lr, 
              weight_decay=wd, 
              momentum=0.9),
              metrics=["accuracy"])

In [111]:
history = model_gru.fit(train_generator_mom, 
                    epochs=20,
                    validation_data=test_generator_mom,
                    shuffle=False,
                    callbacks=[early_stop])

Epoch 1/20
334/334 [==============================] - 18s 29ms/step - loss: 1.0125 - accuracy: 0.4990 - val_loss: 1.0145 - val_accuracy: 0.5417
Epoch 2/20
334/334 [==============================] - 8s 24ms/step - loss: 1.0810 - accuracy: 0.4900 - val_loss: 0.7528 - val_accuracy: 0.5401
Epoch 3/20
334/334 [==============================] - 9s 26ms/step - loss: 1.0195 - accuracy: 0.5079 - val_loss: 0.7475 - val_accuracy: 0.5049
Epoch 4/20
334/334 [==============================] - 9s 26ms/step - loss: 0.9980 - accuracy: 0.4963 - val_loss: 2.0994 - val_accuracy: 0.4577
Epoch 5/20
334/334 [==============================] - 9s 26ms/step - loss: 1.0542 - accuracy: 0.5028 - val_loss: 1.3836 - val_accuracy: 0.5417
Epoch 6/20
334/334 [==============================] - 9s 26ms/step - loss: 0.9842 - accuracy: 0.5214 - val_loss: 0.7610 - val_accuracy: 0.4572
Epoch 7/20
334/334 [==============================] - 9s 26ms/step - loss: 0.9905 - accuracy: 0.5082 - val_loss: 0.9701 - val_accuracy: 0.470

In [112]:
mom_preds = np.where(model_gru.predict(test_generator_mom).flatten(), 1, 0)
true = y_test_mom[0:len(y_test_mom) - win_length ]
np.mean(mom_preds == true)

0.5449620801733478